In [1]:
from pipeline import Pipeline
from lang_pair import LangPair

In [2]:
vi_vi_vocab, vi_en_vocab = Pipeline.load("vi_vi_train_10_chars_10k_vocab").data, Pipeline.load("vi_en_train_10_chars_10k_vocab").data
vi_vi_idxs, vi_en_idxs = Pipeline.load("vi_idx_10chars_filter").data

In [3]:
p = LangPair(vi_vi_idxs, vi_vi_vocab.eos_idx, vi_en_idxs, vi_en_vocab.eos_idx)

In [7]:
p.get_rand_sentence()

(tensor([[5061367],
         [5061157],
         [5061508],
         [5060755],
         [5061502],
         [5061531]]), tensor([[1870123],
         [2722741],
         [2725363],
         [2726168],
         [2726169],
         [2726171]]))